In [41]:
import os, shutil, glob, subprocess

In [6]:
def copy_files_to_single_directory(src, dest):
    for root, dirs, files in os.walk(src):
        for file in files:
            src_file = os.path.join(root, file)
            dest_file = os.path.join(dest, file)
            
            # Check if a file with the same name already exists in the destination
            counter = 1
            base, extension = os.path.splitext(dest_file)
            while os.path.exists(dest_file):
                dest_file = f"{base}-{counter}{extension}"
                counter += 1
            
            shutil.copy2(src_file, dest_file)

In [ ]:
%%time
source_directory = '/Users/nrapstin/Downloads/Fall_2023_photo_dump'
destination_directory = '/Users/nrapstin/Downloads/backup-photos'
copy_files_to_single_directory(source_directory, destination_directory)

In [2]:
def get_files_with_extensions(directory, extensions):
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(directory, f'*{ext}')))
        files.extend(glob.glob(os.path.join(directory, f'*{ext.upper()}')))
    return files

In [5]:
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime

In [3]:
directory = '/Users/nrapstin/Downloads/backup-photos'

In [4]:
img_extensions = [".png", ".jpg", ".jpeg", ".gif"]  # List of extensions

In [7]:
img_list = get_files_with_extensions(directory, img_extensions)

In [8]:
len(img_list)

2787

In [37]:
mov_extensions = [".mov", ".mp4"]  # List of extensions

In [38]:
mov_list = get_files_with_extensions(directory, mov_extensions)

In [39]:
len(mov_list)

464

In [13]:
def get_next_available_filename(filepath):
    directory, filename = os.path.split(filepath)
    file_base, extension = os.path.splitext(filename)

    counter = 1
    new_filename = f"{file_base}-{counter}{extension}"
    new_filepath = os.path.join(directory, new_filename)

    while os.path.exists(new_filepath):
        counter += 1
        new_filename = f"{file_base}-{counter}{extension}"
        new_filepath = os.path.join(directory, new_filename)

    return new_filepath

In [14]:
img_with_meta = {}

# loop over all exported images
for i, img in enumerate(img_list):
    
    # read the image data using PIL
    image = Image.open(img)
    # extract EXIF data
    exifdata = image.getexif()
    
    if exifdata:
        # iterating over all EXIF data fields
        for tag_id in exifdata:

            # get the tag name, instead of human unreadable tag id
            tag = TAGS.get(tag_id, tag_id)
            data = exifdata.get(tag_id)

            # decode bytes 
            if isinstance(data, bytes):
                data = data.decode()
            
            
            # we only care about the date tag
            if tag == 'DateTime':
                date_str = data
                
                # Parse the string into a datetime object
                dt = datetime.strptime(date_str, '%Y:%m:%d %H:%M:%S')
                img_with_meta[i] = { 'img_path' : img, 'dt' : dt}
                


In [15]:
len(img_with_meta)

1759

In [16]:
for key, val in img_with_meta.items():
#     print(key, val)
    # get date
    dt = val['dt']
    
    # Create new directory name based on year-month
    year_folder = os.path.join(directory, str(dt.year))
    month_folder = os.path.join(year_folder, f"{dt.year}-{dt.month:02d}")

    # Create year and year-month directories if they don't exist
    os.makedirs(month_folder, exist_ok=True)
    
    # check if file name corresponds to the date
    img_path_original = val['img_path']
    
    file = img_path_original.split('/')[-1] 
    file, extension = os.path.splitext(file)
    yearmonth = file.split('-')[0]
    year = yearmonth[:4]
    month = yearmonth[4:6]
    year_dt = dt.year
    month_dt = dt.month
    
#     if str(year) != str(year_dt) or str(month) != f"{month_dt:02d}":
#         # if year or month do not match
#         print(key, val)
#         print('date DOES NOT match')
        
    # Create new filename with year, month, day, hour, minute, and second
    base_filename = dt.strftime('%Y%m%d-%H%M%S')
    new_filename = base_filename + extension
    new_file_path = os.path.join(month_folder, new_filename)

    # Check if file exists, and if so, append a digit starting at 1
    if os.path.exists(new_file_path):
        # pick new name
        new_filename = get_next_available_filename(new_file_path)
        new_file_path = os.path.join(month_folder, new_filename)

    print(f"dt: {dt}")
    print(f"img_path_original: {img_path_original}")
    print(f"new_file_path: {new_file_path}")


    # Move the file to the new location
    shutil.move(img_path_original, new_file_path)


dt: 2016-10-24 09:58:18
img_path_original: /Users/nrapstin/Downloads/backup-photos/20161024-095818.jpg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2016/2016-10/20161024-095818.jpg
dt: 2016-10-02 12:44:32
img_path_original: /Users/nrapstin/Downloads/backup-photos/20161002-124432.jpg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2016/2016-10/20161002-124432.jpg
dt: 2016-11-22 07:35:54
img_path_original: /Users/nrapstin/Downloads/backup-photos/20161122-073554.jpg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2016/2016-11/20161122-073554.jpg
dt: 2022-04-19 16:49:08
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220419-164908.jpg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-04/20220419-164908.jpg
dt: 2022-09-02 19:52:11
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220902-195211.jpg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220902-195211.jpg
dt: 2021-12-04 13:16:21
img_path_original: /U

dt: 2022-11-05 14:08:22
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221105-140822.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221105-140822.jpeg
dt: 2022-03-10 10:55:04
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220310-105504.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-03/20220310-105504.jpeg
dt: 2023-04-08 15:35:56
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230408-153556.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230408-153556.jpeg
dt: 2022-11-11 19:41:40
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221111-194140.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221111-194140.jpeg
dt: 2023-02-24 16:30:07
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230224-163007.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-02/20230224-163007.jpeg
dt: 2022-02-07 18:04:07
img_path_or

dt: 2022-09-23 15:21:10
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220923-152110.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220923-152110.jpeg
dt: 2022-12-16 08:05:40
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221216-080540.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-12/20221216-080540.jpeg
dt: 2023-05-06 18:25:10
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230506-182510.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-05/20230506-182510.jpeg
dt: 2022-09-23 13:57:54
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220923-135754.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220923-135754.jpeg
dt: 2023-08-14 18:09:45
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230814-180945.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-08/20230814-180945.jpeg
dt: 2023-10-23 21:08:43
img_path_or

dt: 2023-07-15 01:45:19
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230715-014519.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-07/20230715-014519.jpeg
dt: 2023-04-21 14:58:29
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230421-145829.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230421-145829.jpeg
dt: 2022-06-07 00:09:26
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220607-000926.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220607-000926.jpeg
dt: 2022-03-25 07:57:21
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220325-075721.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-03/20220325-075721.jpeg
dt: 2022-11-16 22:13:49
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221116-221349.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221116-221349.jpeg
dt: 2022-12-18 14:52:44
img_path_or

dt: 2022-08-04 14:50:57
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220804-145057.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220804-145057.jpeg
dt: 2022-01-07 20:32:09
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220107-203209.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-01/20220107-203209.jpeg
dt: 2022-06-27 16:56:57
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220627-165657.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220627-165657.jpeg
dt: 2022-08-19 21:09:52
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220819-210952.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220819-210952.jpeg
dt: 2022-12-10 17:32:02
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221210-173202.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-12/20221210-173202.jpeg
dt: 2022-09-13 09:03:06
img_path_or

dt: 2023-04-08 18:57:31
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230408-185731.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230408-185731.jpeg
dt: 2023-05-05 17:28:15
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230505-172815.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-05/20230505-172815.jpeg
dt: 2023-07-26 20:47:36
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230726-204736.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-07/20230726-204736.jpeg
dt: 2022-08-04 14:59:59
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220804-145959.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220804-145959.jpeg
dt: 2023-09-21 20:42:46
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230921-204246.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-09/20230921-204246.jpeg
dt: 2022-08-04 14:57:35
img_path_or

dt: 2023-04-25 19:07:32
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230425-190732.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230425-190732.jpeg
dt: 2022-12-17 20:25:01
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221217-202501.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-12/20221217-202501.jpeg
dt: 2022-12-16 16:14:19
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221216-161419.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-12/20221216-161419.jpeg
dt: 2022-08-02 08:35:19
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220802-083519-2.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220802-083519-2.jpeg
dt: 2022-11-05 14:08:16
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221105-140816.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221105-140816.jpeg
dt: 2021-12-20 19:13:14
img_pat

In [17]:
# make a new list without metadata
img_list = get_files_with_extensions(directory, img_extensions)

In [18]:
len(img_list)    

1028

In [20]:
def get_creation_date(file_path):
    date_format = "%Y-%m-%d %H:%M:%S"
    
    if file_path.endswith((".png", ".jpg", ".jpeg", ".PNG", ".JPG", ".gif")):
        with Image.open(file_path) as img:
            if 'exif' in img.info:
                exif_data = img._getexif()
                if exif_data and 36867 in exif_data:  # 36867 is the tag for DateTimeOriginal
                    return datetime.strptime(exif_data[36867], '%Y:%m:%d %H:%M:%S')

    return None

In [26]:
# loop over all exported images
for i, img in enumerate(img_list):
    creation_date = get_creation_date(img)
    if creation_date and creation_date > datetime(1970, 1, 1):
        print(i, img)
        
        # Create new directory name based on year-month
        year_folder = os.path.join(directory, str(creation_date.year))
        month_folder = os.path.join(year_folder, f"{creation_date.year}-{creation_date.month:02d}")

        # Create year and year-month directories if they don't exist
        os.makedirs(month_folder, exist_ok=True)
            
        # Create new filename with year, month, day, hour, minute, and second
        file = img.split('/')[-1] 
        file, extension = os.path.splitext(file)
        base_filename = creation_date.strftime('%Y%m%d-%H%M%S')
        new_filename = base_filename + extension
        new_file_path = os.path.join(month_folder, new_filename)
           
        # Check if file exists, and if so, append a digit starting at 1
        if os.path.exists(new_file_path):
            # pick new name
            new_filename = get_next_available_filename(new_file_path)
            new_file_path = os.path.join(month_folder, new_filename)
            
        print(f"creation_date: {creation_date}")
        print(f"img_path_original: {img}")
        print(f"new_file_path: {new_file_path}")
        
        
        # Move the file to the new location
        shutil.move(img, new_file_path)


3 /Users/nrapstin/Downloads/backup-photos/20220809-202307.PNG
creation_date: 2022-08-09 20:23:07
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220809-202307.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220809-202307.PNG
6 /Users/nrapstin/Downloads/backup-photos/20220520-161344.PNG
creation_date: 2022-05-20 16:13:44
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220520-161344.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220520-161344.PNG
8 /Users/nrapstin/Downloads/backup-photos/20220921-064422.PNG
creation_date: 2022-09-21 06:44:22
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220921-064422.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220921-064422.PNG
9 /Users/nrapstin/Downloads/backup-photos/20221013-174331.PNG
creation_date: 2022-10-13 17:43:31
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221013-174331.PNG
new_file_path: /Users/nrapstin/D

93 /Users/nrapstin/Downloads/backup-photos/20220902-135040.PNG
creation_date: 2022-09-02 13:50:40
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220902-135040.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220902-135040.PNG
94 /Users/nrapstin/Downloads/backup-photos/20220606-190402.PNG
creation_date: 2022-06-06 19:04:02
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220606-190402.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220606-190402.PNG
95 /Users/nrapstin/Downloads/backup-photos/20230127-185318.PNG
creation_date: 2023-01-27 18:53:18
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230127-185318.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-01/20230127-185318.PNG
96 /Users/nrapstin/Downloads/backup-photos/20221130-120228.PNG
creation_date: 2022-11-30 12:02:28
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221130-120228.PNG
new_file_path: /Users/nrapst

208 /Users/nrapstin/Downloads/backup-photos/20221118-114254.PNG
creation_date: 2022-11-18 11:42:54
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221118-114254.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221118-114254.PNG
209 /Users/nrapstin/Downloads/backup-photos/20230903-202844.PNG
creation_date: 2023-09-03 20:28:44
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230903-202844.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-09/20230903-202844.PNG
210 /Users/nrapstin/Downloads/backup-photos/20230127-185421.PNG
creation_date: 2023-01-27 18:54:21
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230127-185421.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-01/20230127-185421.PNG
211 /Users/nrapstin/Downloads/backup-photos/20230116-113528.PNG
creation_date: 2023-01-16 11:35:28
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230116-113528.PNG
new_file_path: /Users/nr

328 /Users/nrapstin/Downloads/backup-photos/20221230-233941.PNG
creation_date: 2022-12-30 23:39:41
img_path_original: /Users/nrapstin/Downloads/backup-photos/20221230-233941.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-12/20221230-233941.PNG
329 /Users/nrapstin/Downloads/backup-photos/20220317-155400.PNG
creation_date: 2022-03-17 15:54:00
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220317-155400.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-03/20220317-155400.PNG
332 /Users/nrapstin/Downloads/backup-photos/20230415-223944.PNG
creation_date: 2023-04-15 22:39:44
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230415-223944.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230415-223944.PNG
333 /Users/nrapstin/Downloads/backup-photos/20220921-150758.PNG
creation_date: 2022-09-21 15:07:58
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220921-150758.PNG
new_file_path: /Users/nr

1010 /Users/nrapstin/Downloads/backup-photos/20170816-101840.jpeg
creation_date: 2017-08-16 10:18:40
img_path_original: /Users/nrapstin/Downloads/backup-photos/20170816-101840.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2017/2017-08/20170816-101840.jpeg
1012 /Users/nrapstin/Downloads/backup-photos/20230603-103149.jpeg
creation_date: 2023-06-03 10:31:49
img_path_original: /Users/nrapstin/Downloads/backup-photos/20230603-103149.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-06/20230603-103149.jpeg
1015 /Users/nrapstin/Downloads/backup-photos/20220702-231334.jpeg
creation_date: 2022-07-02 23:13:34
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220702-231334.jpeg
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-07/20220702-231334.jpeg
1018 /Users/nrapstin/Downloads/backup-photos/20220311-171238.jpeg
creation_date: 2022-03-11 17:12:38
img_path_original: /Users/nrapstin/Downloads/backup-photos/20220311-171238.jpeg
new_file_

In [27]:
# make a new list without metadata
img_list = get_files_with_extensions(directory, img_extensions)

In [28]:
len(img_list)

685

In [32]:
os.makedirs(os.path.join(directory, "unsorted"), exist_ok=True)

In [36]:
# move these into unsorted
for i, img in enumerate(img_list):
    
    # Create new filename with year, month, day, hour, minute, and second
    file = img.split('/')[-1] 
    file, extension = os.path.splitext(file)
    base_filename = 'nodate'
    new_filename = base_filename + extension
    new_file_path = os.path.join(os.path.join(directory, "unsorted"), new_filename)

    # Check if file exists, and if so, append a digit starting at 1
    if os.path.exists(new_file_path):
        # pick new name
        new_filename = get_next_available_filename(new_file_path)
        new_file_path = os.path.join(os.path.join(directory, "unsorted"), new_filename)
    
    print(f"img_path_original: {img}")
    print(f"new_file_path: {new_file_path}")
        
    # Move the file to the new location
    shutil.move(img, new_file_path)

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121246.png
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate.png
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115645.png
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-1.png
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114620.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-2.PNG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120234.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-3.PNG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115714.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-4.PNG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114224.PNG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-5.PNG
img_path_original: /Users/nrapstin/Downloads/backup-photos/2023123

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115310.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-58.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121248.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-59.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115648.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-60.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114218.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-61.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115734-1.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-62.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115112.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-63.JPG
img_path_original: /Users/nrapstin/Downloads/backup-phot

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114209.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-170.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114552.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-171.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114208.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-172.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120152.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-173.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115314.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-174.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120350.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-175.JPG
img_path_original: /Users/nrapstin/Downloads/backup-

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120331.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-260.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120325.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-261.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115413.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-262.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114719.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-263.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121211.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-264.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121205.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-265.JPG
img_path_original: /Users/nrapstin/Downloads/backup-

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121550.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-330.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114500.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-331.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120816-1.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-332.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115623.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-333.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121545.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-334.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115810.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-335.JPG
img_path_original: /Users/nrapstin/Downloads/backu

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115208.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-401.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121740.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-402.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121026.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-403.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120338.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-404.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120704-2.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-405.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115432.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-406.JPG
img_path_original: /Users/nrapstin/Downloads/backu

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115544.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-453.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115236.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-454.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114128.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-455.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120514.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-456.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121144.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-457.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120919.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-458.JPG
img_path_original: /Users/nrapstin/Downloads/backup-

img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-115119.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-507.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114207.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-508.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120808.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-509.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121531.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-510.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-121744-1.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-511.JPG
img_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-120230-1.JPG
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate-512.JPG
img_path_original: /Users/nrapstin/Downloads/bac

In [59]:
vid_with_meta = {}
# loop over all exported videos
for i, v in enumerate(mov_list):
    # read the mov metadata
    metadata = subprocess.getoutput("ffmpeg -i %s"  % (v))
    
    # get the date from metadata
    if metadata.find('com.apple.quicktime.creationdate:') != -1:
        date_str = metadata[metadata.find('com.apple.quicktime.creationdate:'):].split("\n")[0].split(": ")[1]
        # Parse the string into a datetime object
        print(f"{i}: {v}")
        dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S%z')
        print(f"date: {dt}")
        
        vid_with_meta[i] = { 'vid_path' : v, 'dt' : dt}


0: /Users/nrapstin/Downloads/backup-photos/20231104-015433.mov
date: 2023-02-19 13:20:32-06:00
2: /Users/nrapstin/Downloads/backup-photos/20211120-153738.mov
date: 2021-11-20 09:36:46-06:00
6: /Users/nrapstin/Downloads/backup-photos/20231104-013948.mov
date: 2022-06-07 03:01:10-05:00
7: /Users/nrapstin/Downloads/backup-photos/20231104-014826.mov
date: 2022-10-01 11:08:48-05:00
9: /Users/nrapstin/Downloads/backup-photos/20211030-145147.mov
date: 2021-10-30 09:51:01-05:00
11: /Users/nrapstin/Downloads/backup-photos/20231104-014651-1.mov
date: 2022-08-27 16:31:28-05:00
12: /Users/nrapstin/Downloads/backup-photos/20211023-122819.mov
date: 2021-10-22 12:08:05-05:00
14: /Users/nrapstin/Downloads/backup-photos/20231104-020235.mov
date: 2023-09-07 21:27:22-05:00
15: /Users/nrapstin/Downloads/backup-photos/20231104-014025.mov
date: 2022-07-03 21:30:04-05:00
16: /Users/nrapstin/Downloads/backup-photos/20231104-013944.mov
date: 2022-06-04 12:02:13-05:00
17: /Users/nrapstin/Downloads/backup-photos

99: /Users/nrapstin/Downloads/backup-photos/20231104-015040.MOV
date: 2022-12-10 17:31:34-06:00
100: /Users/nrapstin/Downloads/backup-photos/20231104-015726.MOV
date: 2023-04-22 15:17:20-05:00
101: /Users/nrapstin/Downloads/backup-photos/20231104-020748.MOV
date: 2023-10-27 19:28:34-05:00
103: /Users/nrapstin/Downloads/backup-photos/20231104-014002.MOV
date: 2022-06-14 11:46:43-05:00
104: /Users/nrapstin/Downloads/backup-photos/20231104-014037-2.MOV
date: 2022-07-08 15:58:38-05:00
105: /Users/nrapstin/Downloads/backup-photos/20231104-020600.MOV
date: 2023-10-14 11:57:17-05:00
106: /Users/nrapstin/Downloads/backup-photos/20231104-015136.MOV
date: 2023-01-03 19:22:48-06:00
107: /Users/nrapstin/Downloads/backup-photos/20231104-020359.MOV
date: 2023-09-28 20:30:22-05:00
108: /Users/nrapstin/Downloads/backup-photos/20231104-014639.MOV
date: 2022-08-20 09:28:07-05:00
109: /Users/nrapstin/Downloads/backup-photos/20231104-015725.MOV
date: 2023-04-22 15:17:17-05:00
110: /Users/nrapstin/Download

185: /Users/nrapstin/Downloads/backup-photos/20231104-015932.MOV
date: 2023-05-17 18:07:06-07:00
186: /Users/nrapstin/Downloads/backup-photos/20231104-020753.MOV
date: 2023-10-28 11:36:35-05:00
187: /Users/nrapstin/Downloads/backup-photos/20231104-014323-1.MOV
date: 2022-08-02 08:38:29-07:00
188: /Users/nrapstin/Downloads/backup-photos/20231104-015501.MOV
date: 2023-03-18 00:45:35-05:00
189: /Users/nrapstin/Downloads/backup-photos/20231104-020343.MOV
date: 2023-09-26 19:09:22-05:00
190: /Users/nrapstin/Downloads/backup-photos/20231104-014019.MOV
date: 2022-06-30 17:45:34-05:00
191: /Users/nrapstin/Downloads/backup-photos/20231104-020141.MOV
date: 2023-08-09 07:23:52-05:00
192: /Users/nrapstin/Downloads/backup-photos/20231104-020814.MOV
date: 2023-10-31 19:39:34-05:00
193: /Users/nrapstin/Downloads/backup-photos/20231104-020828.MOV
date: 2023-11-02 08:22:29-05:00
194: /Users/nrapstin/Downloads/backup-photos/20231104-020632.MOV
date: 2023-10-17 20:43:44-05:00
195: /Users/nrapstin/Downloa

In [60]:
len(vid_with_meta)

243

In [63]:
for key, val in vid_with_meta.items():
#     print(key, val)
    # get date
    dt = val['dt']
    
    # Create new directory name based on year-month
    year_folder = os.path.join(directory, str(dt.year))
    month_folder = os.path.join(year_folder, f"{dt.year}-{dt.month:02d}")

    # Create year and year-month directories if they don't exist
    os.makedirs(month_folder, exist_ok=True)
    
    vid_path_original = val['vid_path']
    
    file = vid_path_original.split('/')[-1] 
    file, extension = os.path.splitext(file)
    yearmonth = file.split('-')[0]
    year = yearmonth[:4]
    month = yearmonth[4:6]
    year_dt = dt.year
    month_dt = dt.month
    
        
    # Create new filename with year, month, day, hour, minute, and second
    base_filename = dt.strftime('%Y%m%d-%H%M%S')
    new_filename = base_filename + extension
    new_file_path = os.path.join(month_folder, new_filename)

    # Check if file exists, and if so, append a digit starting at 1
    if os.path.exists(new_file_path):
        # pick new name
        new_filename = get_next_available_filename(new_file_path)
        new_file_path = os.path.join(month_folder, new_filename)

    print(f"dt: {dt}")
    print(f"vid_path_original: {vid_path_original}")
    print(f"new_file_path: {new_file_path}")


    # Move the file to the new location
    shutil.move(vid_path_original, new_file_path)


dt: 2023-02-19 13:20:32-06:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-015433.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-02/20230219-132032.mov
dt: 2021-11-20 09:36:46-06:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211120-153738.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-11/20211120-093646.mov
dt: 2022-06-07 03:01:10-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-013948.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220607-030110.mov
dt: 2022-10-01 11:08:48-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-014826.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-10/20221001-110848.mov
dt: 2021-10-30 09:51:01-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211030-145147.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-10/20211030-095101.mov
dt: 2022-08-27 

dt: 2022-08-20 09:25:26-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-014433.MOV
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220820-092526.MOV
dt: 2022-08-14 17:56:39-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-014427.MOV
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-08/20220814-175639.MOV
dt: 2022-05-14 13:14:30-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-013837.MOV
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220514-131430.MOV
dt: 2023-10-01 14:10:34-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-020423.MOV
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-10/20231001-141034.MOV
dt: 2022-06-14 11:45:52-05:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-013957.MOV
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220614-114552.MOV
dt: 2022-07-08 

In [64]:
mov_list = get_files_with_extensions(directory, mov_extensions)

In [65]:
len(mov_list)

221

In [77]:
for i, v in enumerate(mov_list):
#     print(f"{i}: {v}")
    # read the mov metadata
    metadata = subprocess.getoutput("ffmpeg -i %s"  % (v))
    
    # get the date from metadata
    if metadata.find('creation_time') != -1:
        date_str = metadata[metadata.find('creation_time'):].split("\n")[0].split(": ")[1]
        # Parse the string into a datetime object
        date_str = date_str.replace('Z', '+0000')
    
        dt = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%f%z')
#         print(f"date: {dt}")
        
        # check if file name corresponds to the date
#         img_path_original = val['img_path']

        file = v.split('/')[-1] 
        file, extension = os.path.splitext(file)
        yearmonth = file.split('-')[0]
        year = yearmonth[:4]
        month = yearmonth[4:6]
        
        year_dt = dt.year
        month_dt = dt.month
        
        if str(year) != str(year_dt) or str(month) != f"{month_dt:02d}":
            # if year or month do not match
            print(i, v)
            print('date DOES NOT match')
        
        # Create new directory name based on year-month
        year_folder = os.path.join(directory, str(dt.year))
        month_folder = os.path.join(year_folder, f"{dt.year}-{dt.month:02d}")

        # Create year and year-month directories if they don't exist
        os.makedirs(month_folder, exist_ok=True)
    
        # Create new filename with year, month, day, hour, minute, and second
        base_filename = dt.strftime('%Y%m%d-%H%M%S')
        new_filename = base_filename + extension
        new_file_path = os.path.join(month_folder, new_filename)

        # Check if file exists, and if so, append a digit starting at 1
        if os.path.exists(new_file_path):
            # pick new name
            new_filename = get_next_available_filename(new_file_path)
            new_file_path = os.path.join(month_folder, new_filename)

        print(f"dt: {dt}")
        print(f"vid_path_original: {v}")
        print(f"new_file_path: {new_file_path}")


        # Move the file to the new location
        shutil.move(v, new_file_path)

dt: 2021-10-12 15:31:38+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211012-153138.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-10/20211012-153138.mov
dt: 2021-10-29 14:51:09+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211029-145109.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-10/20211029-145109.mov
dt: 2022-06-10 15:34:53+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220610-153453.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-06/20220610-153453.mov
dt: 2021-10-07 17:27:38+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211007-172738.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-10/20211007-172738.mov
dt: 2022-05-04 16:24:23+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220504-162423.mov
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220504-162423.mov
dt: 2021-10-28 

dt: 2023-04-29 21:00:11+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230429-210011.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230429-210011.mp4
dt: 2023-11-04 01:48:18+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-014818.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-014818.mp4
dt: 2023-05-06 15:13:07+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230506-151307.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-05/20230506-151307.mp4
dt: 2023-01-28 18:47:08+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230128-184708.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-01/20230128-184708.mp4
dt: 2023-11-04 02:04:05+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-020405.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-020405.mp4
dt: 2021-11-25 

dt: 2021-11-11 00:50:13+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211111-005013.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-11/20211111-005013.mp4
dt: 2023-03-24 22:33:37+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230324-223337.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-03/20230324-223337.mp4
dt: 2023-11-04 01:51:47+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-015147.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-015147.mp4
dt: 2023-11-04 02:05:13+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-020513.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-020513.mp4
dt: 2022-09-18 06:50:05+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220918-065005.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-09/20220918-065005.mp4
dt: 2023-11-04 

dt: 2023-10-08 21:11:01+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231008-211101.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-10/20231008-211101.mp4
dt: 2022-01-23 20:50:18+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220123-205018.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-01/20220123-205018.mp4
dt: 2022-05-21 15:17:12+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220521-151712.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220521-151712.mp4
dt: 2022-05-21 15:48:59+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220521-154859.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220521-154859.mp4
dt: 2021-10-02 23:42:20+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20211002-234220.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2021/2021-10/20211002-234220.mp4
dt: 2022-03-27 

dt: 2022-05-03 16:22:59+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220503-162259.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220503-162259.mp4
dt: 2023-05-28 00:07:44+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230528-000744.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-05/20230528-000744.mp4
dt: 2023-04-29 22:10:17+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230429-221017.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230429-221017.mp4
dt: 2023-04-29 12:31:09+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230429-123109.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230429-123109.mp4
dt: 2023-11-04 02:00:33+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-020033.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-020033.mp4
dt: 2023-11-04 

dt: 2023-11-04 01:51:06+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-015106.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-015106.mp4
dt: 2023-04-29 20:49:06+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20230429-204906.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-04/20230429-204906.mp4
dt: 2022-11-04 22:50:45+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20221104-225045.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-11/20221104-225045.mp4
dt: 2022-05-21 15:44:30+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20220521-154430.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2022/2022-05/20220521-154430.mp4
dt: 2023-11-04 02:00:23+00:00
vid_path_original: /Users/nrapstin/Downloads/backup-photos/20231104-020023.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/2023/2023-11/20231104-020023.mp4
dt: 2023-11-04 

In [78]:
mov_list = get_files_with_extensions(directory, mov_extensions)

In [80]:
file = mov_list[0]

In [81]:
metadata = subprocess.getoutput("ffmpeg -i %s"  % (file))

In [82]:
metadata

"ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers\n  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)\n  configuration: --prefix=/usr/local/Cellar/ffmpeg/6.1.1_1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libso

In [83]:
from hachoir.parser import createParser
from hachoir.metadata import extractMetadata

In [84]:
parser = createParser(file)
metadata = extractMetadata(parser)
creation_date = metadata.get('creation_date')

In [85]:
creation_date

datetime.datetime(1904, 1, 1, 0, 0)

In [86]:
mov_list

['/Users/nrapstin/Downloads/backup-photos/20231230-114751.mp4']

In [89]:
# move these into unsorted
for i, v in enumerate(mov_list):
    
    # Create new filename with year, month, day, hour, minute, and second
    file = v.split('/')[-1] 
    file, extension = os.path.splitext(file)
    base_filename = 'nodate'
    new_filename = base_filename + extension
    new_file_path = os.path.join(os.path.join(directory, "unsorted"), new_filename)

    # Check if file exists, and if so, append a digit starting at 1
    if os.path.exists(new_file_path):
        # pick new name
        new_filename = get_next_available_filename(new_file_path)
        new_file_path = os.path.join(os.path.join(directory, "unsorted"), new_filename)
    
    print(f"v_path_original: {v}")
    print(f"new_file_path: {new_file_path}")
        
    # Move the file to the new location
    shutil.move(v, new_file_path)

v_path_original: /Users/nrapstin/Downloads/backup-photos/20231230-114751.mp4
new_file_path: /Users/nrapstin/Downloads/backup-photos/unsorted/nodate.mp4
